In [96]:
import pandas as pd

In [172]:
df1 = pd.read_csv('data/movie/tmdb_5000_credits.csv')
df2 = pd.read_csv('data/movie/tmdb_5000_movies.csv')

In [173]:
df1.rename(columns={'movie_id':'id'}, inplace=True)
df1.columns

Index(['id', 'title', 'cast', 'crew'], dtype='object')

In [174]:
df_temp = df1[['id', 'cast', 'crew']]
df_temp.columns

Index(['id', 'cast', 'crew'], dtype='object')

In [175]:
df = df2.merge(df_temp, on='id')
df.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'cast', 'crew'],
      dtype='object')

In [101]:
df['overview'].head(5)

0    In the 22nd century, a paraplegic Marine is di...
1    Captain Barbossa, long believed to be dead, ha...
2    A cryptic message from Bond’s past sends him o...
3    Following the death of District Attorney Harve...
4    John Carter is a war-weary, former military ca...
Name: overview, dtype: object

In [102]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')

In [103]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
ENGLISH_STOP_WORDS

frozenset({'a',
           'about',
           'above',
           'across',
           'after',
           'afterwards',
           'again',
           'against',
           'all',
           'almost',
           'alone',
           'along',
           'already',
           'also',
           'although',
           'always',
           'am',
           'among',
           'amongst',
           'amoungst',
           'amount',
           'an',
           'and',
           'another',
           'any',
           'anyhow',
           'anyone',
           'anything',
           'anyway',
           'anywhere',
           'are',
           'around',
           'as',
           'at',
           'back',
           'be',
           'became',
           'because',
           'become',
           'becomes',
           'becoming',
           'been',
           'before',
           'beforehand',
           'behind',
           'being',
           'below',
           'beside',
           'besides'

In [104]:
df['overview'].isnull().sum()

np.int64(3)

In [105]:
df['overview'].fillna('', inplace=True)

C:\Users\KOSMO\AppData\Local\Temp\ipykernel_13000\2799063249.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['overview'].fillna('', inplace=True)


In [106]:
df['overview'].isnull().sum()

np.int64(0)

In [107]:
len(df['overview'])

4803

In [108]:
#BOW 생성
tfidf_matrix = tfidf.fit_transform(df['overview'])

In [109]:
tfidf_matrix.shape

(4803, 20978)

In [110]:
#문장 유사도
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [111]:
cosine_sim.shape

(4803, 4803)

In [112]:
cosine_sim

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.02160533, 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.01488159, 0.        ,
        0.        ],
       ...,
       [0.        , 0.02160533, 0.01488159, ..., 1.        , 0.01609091,
        0.00701914],
       [0.        , 0.        , 0.        , ..., 0.01609091, 1.        ,
        0.01171696],
       [0.        , 0.        , 0.        , ..., 0.00701914, 0.01171696,
        1.        ]], shape=(4803, 4803))

In [113]:
df['title'].head()

0                                      Avatar
1    Pirates of the Caribbean: At World's End
2                                     Spectre
3                       The Dark Knight Rises
4                                 John Carter
Name: title, dtype: object

In [114]:
title='John Carter'
idx=df[df['title']==title].index[0]
idx

np.int64(4)

In [115]:
cosine_sim[4]

array([0.        , 0.03336868, 0.        , ..., 0.00612609, 0.        ,
       0.        ], shape=(4803,))

In [116]:
test_consine_sim = list(enumerate(cosine_sim[4]))
test_consine_sim = sorted(test_consine_sim, key=lambda x:x[1], reverse=True)
test_consine_sim=test_consine_sim[1:11]
index = [i[0] for i in test_consine_sim]
index

[1254, 4161, 2932, 3349, 1307, 3068, 345, 581, 2998, 4274]

In [117]:
df.loc[index, 'title']

1254                          Get Carter
4161         The Marine 4: Moving Target
2932                        Raising Cain
3349                           Desperado
1307                       The Hurricane
3068                         Rescue Dawn
345                          Rush Hour 2
581              Star Trek: Insurrection
2998                               Devil
4274    Eddie: The Sleepwalking Cannibal
Name: title, dtype: object

In [127]:
def def_cosine_sim():
    #BOW생성
    from sklearn.feature_extraction.text import TfidfVectorizer
    tfidf = TfidfVectorizer(stop_words='english')
    
    df = pd.read_csv('data/movie/tmdb_5000_movies.csv')
    df.fillna({'overview':''}, inplace=True)
    tfidf_matrix = tfidf.fit_transform(df['overview'])

    #문장 유사도
    from sklearn.metrics.pairwise import linear_kernel
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
    return cosine_sim

In [128]:
cosine_sim = def_cosine_sim()
cosine_sim

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.02160533, 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.01488159, 0.        ,
        0.        ],
       ...,
       [0.        , 0.02160533, 0.01488159, ..., 1.        , 0.01609091,
        0.00701914],
       [0.        , 0.        , 0.        , ..., 0.01609091, 1.        ,
        0.01171696],
       [0.        , 0.        , 0.        , ..., 0.00701914, 0.01171696,
        1.        ]], shape=(4803, 4803))

In [129]:
df[['title']].head()

,title
0,Avatar
1,Pirates of the Caribbean: At World's End
2,Spectre
3,The Dark Knight Rises
4,John Carter


In [130]:
title='The Dark Knight Rises'
idx=df[df['title']==title].index[0]
print(idx)

3


In [131]:
sim = cosine_sim[idx]
title_sim = list(enumerate(sim))
title_sim = sorted(title_sim, key=lambda x:x[1], reverse=True)
title_sim = title_sim[1:11]
index = [x[0] for x in title_sim]
df.loc[index, 'title']

65                              The Dark Knight
299                              Batman Forever
428                              Batman Returns
1359                                     Batman
3854    Batman: The Dark Knight Returns, Part 2
119                               Batman Begins
2507                                  Slow Burn
9            Batman v Superman: Dawn of Justice
1181                                        JFK
210                              Batman & Robin
Name: title, dtype: object

In [141]:
def recommend(title):
    import pickle

    df = pd.read_csv('data/movie/tmdb_5000_movies.csv')
    idx=df[df['title']==title].index[0]

    cosine_sim = pickle.load(open('data/movie/cosine_sim.pickle', 'rb'))
    sim = cosine_sim[idx]

    sim = list(enumerate(sim))
    sim = sorted(sim, key=lambda x: x[1], reverse=True)
    sim = sim[1:11]
    index = [x[0] for x in sim]
    return index

In [142]:
index = recommend('Avatar')
index

[3604, 2130, 634, 1341, 529, 1610, 311, 847, 775, 2628]

In [230]:
df.loc[index, ['id', 'title']]

,id,title
14,49521,Man of Steel
813,1924,Superman
870,8536,Superman II
9,209112,Batman v Superman: Dawn of Justice
46,127585,X-Men: Days of Future Past
1139,46528,The Warrior's Way
1296,9531,Superman III
2433,11411,Superman IV: The Quest for Peace
0,19995,Avatar
1,285,Pirates of the Caribbean: At World's End


In [135]:
idx = recommend('Batman Forever')
df.loc[idx, 'title']

3                         The Dark Knight Rises
119                               Batman Begins
65                              The Dark Knight
428                              Batman Returns
210                              Batman & Robin
3854    Batman: The Dark Knight Returns, Part 2
1359                                     Batman
4343                                   Cry_Wolf
174                         The Incredible Hulk
9            Batman v Superman: Dawn of Justice
Name: title, dtype: object

In [137]:
cosine_sim = def_cosine_sim()
cosine_sim[:3]

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.02160533, 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.01488159, 0.        ,
        0.        ]], shape=(3, 4803))

In [144]:
import pickle
pickle.dump(cosine_sim, open('data/movie/cosine_sim.pickle', 'wb'))

In [179]:
merge = df.copy()

In [180]:
#장느, 키워드, 주연배우 , 감독
merge.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'cast', 'crew'],
      dtype='object')

In [181]:
from ast import literal_eval
merge['genres'] = merge['genres'].apply(literal_eval)

In [183]:
g=merge.loc[0, 'genres']
type(g)

list

In [184]:
cols = ['cast', 'crew', 'keywords']
for col in cols:
    merge[col] = merge[col].apply(literal_eval) 

In [187]:
import numpy as np
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [188]:
get_director(merge.loc[0, 'crew'])

'James Cameron'

In [189]:
merge['director'] = merge['crew'].apply(get_director)

In [190]:
merge['director'].head()

0        James Cameron
1       Gore Verbinski
2           Sam Mendes
3    Christopher Nolan
4       Andrew Stanton
Name: director, dtype: object

In [191]:
merge['director'].isnull().sum()

np.int64(30)

In [192]:
merge.fillna({'director':''}, inplace=True)

In [193]:
merge['director'].isnull().sum()

np.int64(0)

In [198]:
merge.loc[0, 'genres']

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [199]:
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        if len(names)>3:
            names = names[:3]
        return names
    return []

In [200]:
cols = ['cast', 'keywords', 'genres']
for col in cols:
    merge[col] = merge[col].apply(get_list)

In [202]:
merge[['title', 'director', 'cast', 'keywords', 'genres']].head()

,title,director,cast,keywords,genres
0,Avatar,James Cameron,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[culture clash, future, space war]","[Action, Adventure, Fantasy]"
1,Pirates of the Caribbean: At World's End,Gore Verbinski,"[Johnny Depp, Orlando Bloom, Keira Knightley]","[ocean, drug abuse, exotic island]","[Adventure, Fantasy, Action]"
2,Spectre,Sam Mendes,"[Daniel Craig, Christoph Waltz, Léa Seydoux]","[spy, based on novel, secret agent]","[Action, Adventure, Crime]"
3,The Dark Knight Rises,Christopher Nolan,"[Christian Bale, Michael Caine, Gary Oldman]","[dc comics, crime fighter, terrorist]","[Action, Crime, Drama]"
4,John Carter,Andrew Stanton,"[Taylor Kitsch, Lynn Collins, Samantha Morton]","[based on novel, mars, medallion]","[Action, Adventure, Science Fiction]"


In [203]:
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i).replace(' ', '') for i in x]
    else:
        if isinstance(x, str):
            return str.lower(x).replace(' ','')
        else:
            return ''

In [204]:
cols = ['director', 'cast', 'keywords', 'genres']
for col in cols:
    merge[col] = merge[col].apply(clean_data)

In [205]:
merge[['title', 'director', 'cast', 'keywords', 'genres']].head()

,title,director,cast,keywords,genres
0,Avatar,jamescameron,"[samworthington, zoesaldana, sigourneyweaver]","[cultureclash, future, spacewar]","[action, adventure, fantasy]"
1,Pirates of the Caribbean: At World's End,goreverbinski,"[johnnydepp, orlandobloom, keiraknightley]","[ocean, drugabuse, exoticisland]","[adventure, fantasy, action]"
2,Spectre,sammendes,"[danielcraig, christophwaltz, léaseydoux]","[spy, basedonnovel, secretagent]","[action, adventure, crime]"
3,The Dark Knight Rises,christophernolan,"[christianbale, michaelcaine, garyoldman]","[dccomics, crimefighter, terrorist]","[action, crime, drama]"
4,John Carter,andrewstanton,"[taylorkitsch, lynncollins, samanthamorton]","[basedonnovel, mars, medallion]","[action, adventure, sciencefiction]"


In [208]:
def create_soup(x):
    str = x['director'] + ' '
    str += ' '.join(x['keywords']) + ' '
    str += ' '.join(x['cast']) + ' '
    str += ' '.join(x['genres']) + ' '
    return str

In [209]:
merge['soup'] = merge.apply(create_soup, axis=1)

In [213]:
merge['soup'].head()

0    jamescameron cultureclash future spacewar samw...
1    goreverbinski ocean drugabuse exoticisland joh...
2    sammendes spy basedonnovel secretagent danielc...
3    christophernolan dccomics crimefighter terrori...
4    andrewstanton basedonnovel mars medallion tayl...
Name: soup, dtype: object

In [215]:
merge.to_csv('data/movie/movies.csv', index=False)

In [212]:
merge['overview']

0       In the 22nd century, a paraplegic Marine is di...
1       Captain Barbossa, long believed to be dead, ha...
2       A cryptic message from Bond’s past sends him o...
3       Following the death of District Attorney Harve...
4       John Carter is a war-weary, former military ca...
                              ...                        
4798    El Mariachi just wants to play his guitar and ...
4799    A newlywed couple's honeymoon is upended by th...
4800    "Signed, Sealed, Delivered" introduces a dedic...
4801    When ambitious New York attorney Sam is sent t...
4802    Ever since the second grade when he first saw ...
Name: overview, Length: 4803, dtype: object

In [216]:
df_movie = pd.read_csv('data/movie/movies.csv')
df_movie['soup']

0       jamescameron cultureclash future spacewar samw...
1       goreverbinski ocean drugabuse exoticisland joh...
2       sammendes spy basedonnovel secretagent danielc...
3       christophernolan dccomics crimefighter terrori...
4       andrewstanton basedonnovel mars medallion tayl...
                              ...                        
4798    robertrodriguez unitedstates–mexicobarrier leg...
4799    edwardburns  edwardburns kerrybishé marshadiet...
4800    scottsmith date loveatfirstsight narration eri...
4801     danielhsia  danielhenney elizacoupe billpaxton  
4802    brianherzlinger obsession camcorder crush drew...
Name: soup, Length: 4803, dtype: object

In [217]:
def def_cosine_sim2():
    #BOW생성
    from sklearn.feature_extraction.text import CountVectorizer
    couter = CountVectorizer(stop_words='english')
    
    df = pd.read_csv('data/movie/movies.csv')
    df.fillna({'soup':''}, inplace=True)
    couter_matrix = couter.fit_transform(df['soup'])

    #문장 유사도
    from sklearn.metrics.pairwise import linear_kernel
    cosine_sim = linear_kernel(couter_matrix, couter_matrix)
    return cosine_sim

In [222]:
import pickle
pickle.dump(cosine_sim, open('data/movie/cosine_sim.pickle', 'wb'))

In [221]:
cosine_sim2 = def_cosine_sim2()
pickle.dump(cosine_sim2, open('data/movie/cosine_sim2.pickle', 'wb'))


In [225]:
def recommend2(title):
    import pickle

    df = pd.read_csv('data/movie/tmdb_5000_movies.csv')
    idx=df[df['title']==title].index[0]

    cosine_sim = pickle.load(open('data/movie/cosine_sim2.pickle', 'rb'))
    sim = cosine_sim[idx]

    sim = list(enumerate(sim))
    sim = sorted(sim, key=lambda x: x[1], reverse=True)
    sim = sim[1:11]
    index = [x[0] for x in sim]
    return index

In [228]:
index = recommend2('Superman Returns')
df.loc[index, 'title']

14                                  Man of Steel
813                                     Superman
870                                  Superman II
9             Batman v Superman: Dawn of Justice
46                    X-Men: Days of Future Past
1139                           The Warrior's Way
1296                                Superman III
2433            Superman IV: The Quest for Peace
0                                         Avatar
1       Pirates of the Caribbean: At World's End
Name: title, dtype: object

In [232]:
df = pd.read_csv('data/movie/movies.csv')
df.loc[0, ['id', 'title']]

id        19995
title    Avatar
Name: 0, dtype: object

In [242]:
from tmdbv3api import Movie, TMDb
tmdb = TMDb()
tmdb.api_key='c668cda4cf75bf267ef2aeffa2da0341'
tmdb.language='ko-KR'
movie = Movie()

details = movie.details('19995')
#print(details)
title=details['title']
poster = 'https://image.tmdb.org/t/p/w500' + details['poster_path']
overview = details['overview']
print(title)
print(poster)
print(overview)

아바타
https://image.tmdb.org/t/p/w500/m5lCha2XcbDowDoYHPc0DTNaCPU.jpg
가까운 미래, 지구는 에너지 고갈 문제를 해결하기 위해 머나먼 행성 판도라에서 대체 자원을 채굴하기 시작한다. 하지만 판도라의 독성을 지닌 대기로 인해 자원 획득에 어려움을 겪게 된 인류는 판도라의 토착민 나비의 외형에 인간의 의식을 주입, 원격 조종이 가능한 새로운 생명체를 탄생시키는 프로그램을 개발한다. 한편 하반신이 마비된 전직 해병대원 제이크 설리는 아바타 프로그램에 참가할 것을 제안받는다. 그 곳에서 자신의 아바타를 통해 자유롭게 걸을 수 있게 된 제이크는 자원 채굴을 막으려는 나비의 무리에 침투하라는 임무를 부여받는데...
